In [1]:
import argparse
import os.path
import time
import math
import numpy
import pylab
import scipy.stats
import matplotlib
matplotlib.rc('mathtext', fontset='stixsans', default='regular')
import re
import rmgpy
from rmgpy.quantity import constants
from rmgpy.kinetics import Arrhenius, ArrheniusEP, KineticsData
from rmgpy.data.base import getAllCombinations
from autotst.database import *
from rmgpy.species import Species
from rmgpy.data.rmg import RMGDatabase
import logging

In [3]:
group = """
multiplicity [1,2,3,4,5]
1 *1 Cs u0 {2,S} {3,S} {4,S} {5,S}
2 *2 H  u0 {1,S}
3    H  u0 {1,S}
4    H  u0 {1,S}
5    Cs ux {1,S}
"""
i = Group().fromAdjacencyList(group)
type(i)

rmgpy.molecule.group.Group

In [4]:
import numpy as np
np.array()

TypeError: Required argument 'object' (pos 1) not found

In [5]:
x.database.groups.loadEntry(31, 'CsRRH', """
multiplicity [1,2,3,4,5]
1 *1 Cs  u0 {2,S} {3,S} {4,S} {5,S}
2 *2 H   u0 {1,S}
3    R!H ux {1,S}
4    R!H ux {1,S}
5    R!H ux {1,S}
""')

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-5-bbb1c6d40b91>, line 8)

In [6]:
def TS_Database_Update(families, auto_save = False):
    """
    Loads RMG Databse,
    Creaes instance of TS_updater for each reaction family in families,
    Return dictionary of family:family's instance of the updater
    """
    
    assert isinstance(families, list), "Families must be a list. If singular family, still keep it in list"
    acceptable_families = os.listdir(os.path.join(os.path.expandvars("$RMGpy"), "..", "AutoTST", "database"))
    for family in families:
        assert isinstance(family, str), "Family names must be provided as strings"
        if family.upper() not in (family.upper() for family in acceptable_families):
            logging.warning('"{}" is not a known Kinetics Family'.format(family))
            families.remove(family)
    
    logging.info("Loading RMG Database...")
    rmg_database = RMGDatabase()
    database_path = os.path.join(os.path.expandvars('$RMGpy'), "..",  'RMG-database', 'input')
    
    try:
        rmg_database.load(database_path,
                         #kineticsFamilies=['H_Abstraction'],
                         kineticsFamilies=families,
                         transportLibraries=[],
                         reactionLibraries=[],
                         seedMechanisms=[],
                         thermoLibraries=['primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'CBS_QB3_1dHR' ],
                         solvation=False,
                         )
    except:
        logging.error("Failed to Load RMG Database at {}".format(database_path))
    
    Databases = {family:TS_Updater(family, rmg_database) for family in families}
    
    if auto_save == True:
        save_all_individual_databases(Databases)
    
    return Databases

def save_all_individual_databases(Databases):
    """
    To save all TS_updater instances by means of the dict supplied by TS_Database_Update()
    """
    for family, database in Databases:
        database.save_database()
    return
    
################################################################################################

class TS_Updater:
    """
    Class for use in updating TS training databases
    
    Attributes:
    self.family                 : Relavent Reaction Family
    self.path                   : Path to family
    self.database               : Source for TS geometries
    self.training_set           : Lists of Reaction and corresponding TS Geometries
    
    self.top_nodes              : The two top nodes of the tree of related structures
    self.all_entries            : All the nodes in the tree
    
    self.direct_groups          : Group that is directly matched with reactant structure
    self.nodes_to_update        : Unique list of direct groups and their ancestors
    self.group_ancestors        : Dict organized by {direct group: list of direct groups and its ancestors}
    self.reaction_templates     : The two groups associated with a given reaction's reactants, organized by reaction
    
    self.groupComments          : Templates that are relavent to that entry
    self.groupCounts            : Number of relavant combinations of groups that contribute to that entry
    self.groupUncertainties     : Uncertainty in the optimized TS geometry for that node/entry
    self.groupValues            : Optimized TS geometry for that node/entry

    self.A                      : Binary Matrix (all combinations of those relavent groups for all reactions) by (relavant groups + 1)
    self.b                      : Ax=b, x is found, b is (all combinations of relavent groups for all reactions) by (3 distances) 
    """
    
    def __init__(self, family, rmg_database, path = None):
        
        if path is not None:
            self.path = path
        else:
            self.path = os.path.join(os.path.expandvars("$RMGpy"), "..", "AutoTST", "database", family)
            
        self.family = family
        
        self.set_TS_training_data(rmg_database)
        
        self.update_indices()
        self.set_group_info()
        self.initialize_entry_attributes()
        self.adjust_distances()
        self.set_entry_data()

    
    def set_TS_training_data(self, rmg_database):
        """
        Loads Database, sets it as class attribute, sets training_set from database
        """
        from autotst.database import DistanceData, TransitionStateDepository, TSGroups, TransitionStates
        ts_database = TransitionStates()
        #path = os.path.join(os.path.expandvars("$RMGpy"), "..", "AutoTST", "database", self.family)
        path = self.path
        global_context = { '__builtins__': None }
        local_context={'DistanceData': DistanceData}
        assert self.family in rmg_database.kinetics.families.keys(), "{} not found in kinetics families. Could not Load".format(family)
        family = rmg_database.kinetics.families[self.family]
        ts_database.family = family
        ts_database.load(path, local_context, global_context)
        self.database = ts_database
        # Reaction must be a template reaction... found above
        
        logging.info("Getting Training Data for {}".format(family))
        training_data = [ (entry.item, entry.data.distances) for entry in list(ts_database.depository.entries.itervalues())]
        self.training_set = training_data
        logging.info("Total Distances Count: {}".format(len(self.training_set)))
        return
    
    
    def update_indices(self):
        """
        Updating entry indices based off of tree indices, tree indices are found by descending the tree
        Without this, indices will be based off of previous database which may not be aligned by the current tree
        """
        all_entries = []
        self.top_nodes = self.database.groups.top
        assert len(self.top_nodes) == 2, 'Only set to work for trees with two top nodes. It has: {}'.format(len(self.top_nodes))
        
        for top_node in self.top_nodes:
            descendants = [top_node] + self.database.groups.descendants(top_node)
            all_entries.extend(descendants)

        for tree_index, entry in enumerate(all_entries):
            #tree_indices[entry] = tree_index
            self.database.groups.entries[entry.label].index = tree_index
            entry.index = tree_index
        
        self.all_entries = all_entries
        logging.info("Updating Indices based off of Tree...")
        logging.info("Tree size: {}".format(len(all_entries)))
        return


    def set_group_info(self):
        """
        Sets useful group info that is used by further class methods
        """
        
        #Direct groups are the lowest level node that matches the reactant structure
        direct_groups = []
        all_reactant_groups = {} #the two groups (template) of the reactants organized by reactions

        for reaction, distance_data in self.training_set:
            reactant_groups = [] #The groups that represent each reactant - also known as the template
            """    
            for reactant in reaction.reactants:
                reactant = reactant.molecule[0]

                atoms = list(reactant.getLabeledAtoms().itervalues())
                assert atoms is not None

                for top_node in self.top_nodes:
                    temp_group = self.database.groups.descendTree(reactant, atoms, root=top_node)
                    if temp_group is not None:     #Temp_group will only be found using one of the two top_nodes
                        reactant_group = temp_group
                        break

                assert reactant_group is not None

                if isinstance(reactant_group, str):
                    assert False, "Versioning control problem: This should be a redundant check, but clearly is not in this case"
                    reactant_group = ts_database.groups.entries[reactant_group]
                assert isinstance(reactant_group, Entry)
            """
    
            for top_node in self.top_nodes:

                for reactant in reaction.reactants:
                    if isinstance(reactant, rmgpy.species.Species):
                        reactant = reactant.molecule[0]

                    atoms = list(reactant.getLabeledAtoms().itervalues())
                    assert atoms is not None

                    #temp_group = self.database.groups.descendTree(reactant, atoms, root=top_node)
                    temp_group = self.database.groups.descendTree(reactant, atoms, root=top_node)
                    if temp_group is not None:     #Temp_group will only be found using one of the two top_nodes
                        reactant_group = temp_group
                        break


                reactant_groups.append(reactant_group)        
                direct_groups.append(reactant_group)

            all_reactant_groups[reaction] = reactant_groups #storing the templates by reaction

        direct_groups = list(set(direct_groups))
        direct_groups.sort(key=lambda x:x.index)

        all_ancestors = {} #Key is group, value is its itself and all its ancestors
        for direct_group in direct_groups:
            ancestors = [direct_group] + self.database.groups.ancestors(direct_group)
            for ancestor in ancestors:
                if ancestor in all_ancestors.keys():
                    continue
                else:
                    all_ancestors[ancestor] = [ancestor] + self.database.groups.ancestors(ancestor)

        # We need a list of unique nodes that are directly involved in a reaction or the ancestor of a group that is
        nodes_to_update = [group for group in all_ancestors if group not in self.top_nodes]
        nodes_to_update.sort(key=lambda x:x.index)

        #Group info that is needed to simplify following methods
        self.direct_groups = direct_groups
        self.nodes_to_update = nodes_to_update
        self.group_ancestors = all_ancestors
        self.reaction_templates = all_reactant_groups
        
        logging.info('Nodes to Update: {}'.format(len(self.nodes_to_update)))
        logging.info("Reaction Templates: {}".format(len(self.reaction_templates)))
        return
    
    
    def initialize_entry_attributes(self):
        """
        Attributes of each entry, initializing to size of all_entries
        """
        self.groupComments = {}; self.groupCounts = {}; self.groupUncertainties = {}; self.groupValues = {}
        for entry in self.all_entries:
            self.groupComments[entry] = set()
            self.groupCounts[entry] = []
            self.groupUncertainties[entry] = []
            self.groupValues[entry] = []
        return
    
    def adjust_distances(self):
        """
        Creating A and b of Ax=b
        """
        def getAllCombinations(nodeLists):
            """
            From base.py:
            Generate a list of all possible combinations of items in the list of
            lists `nodeLists`. Each combination takes one item from each list
            contained within `nodeLists`. The order of items in the returned lists
            reflects the order of lists in `nodeLists`. For example, if `nodeLists` was
            [[A, B, C], [N], [X, Y]], the returned combinations would be
            [[A, N, X], [A, N, Y], [B, N, X], [B, N, Y], [C, N, X], [C, N, Y]].
            """

            items = [[]]
            for nodeList in nodeLists:
                items = [ item + [node] for node in nodeList for item in items ]

            return items
        ##############################################
    
        distance_keys = sorted(self.training_set[0][1].keys())
        #distance_keys are ['d12', 'd13', 'd23']

        A = []
        b = []
        for reaction, distance_data in self.training_set:
            template = self.reaction_templates[reaction]
            distances_list = [distance_data[key] for key in distance_keys]

            relavent_combinations = []
            for reactant_group in template:
                relavent_combinations.append(self.group_ancestors[reactant_group])
            assert len(relavent_combinations) == 2 #will throw if reaction does not have 2 reactants

            relavent_combinations = getAllCombinations(relavent_combinations)
            #rel_comb is just all combinations of reactant1 and its ancestors with reactant2 and its ancestors

            for combination in relavent_combinations:
                Arow = [1 if group in combination else 0 for group in self.nodes_to_update]
                Arow.append(1) #For use in finding the family component
                #Arow is a binary vector of len(groupList)+1 representing contributing groups to this reaction's distance data
                A.append(Arow)
                b.append(distances_list)
                for group in combination:
                    if isinstance(group, str):
                        assert False, "Discrepancy between versions of RMG_Database and this one"

                    self.groupComments[group].add('{0!s}'.format(template))

        self.A = numpy.array(A)
        self.b = numpy.array(b)
        return
    
    def set_entry_data(self):
        """
        Using A and b to find stats for relavent nodes of tree
        """
        import scipy.stats
        # Groups M and N are associated with a reaction that has a known ts geometry
        # Groups M, N, and the family component must add together to get as close to that geometry as possible
        # M and N are optimized based off of all reactionas they are involved with and the family component is optimized over all reactions of that family


        distance_keys = sorted(self.training_set[0][1].keys())
        #distance_keys are ['d12', 'd13', 'd23']

        x, residuals, rank, s = numpy.linalg.lstsq(self.A, self.b)
        for i, distance_key in enumerate(distance_keys):
            # Determine error in each group
            variance_sums = numpy.zeros(len(self.nodes_to_update)+1, numpy.float64)
            stdev = numpy.zeros(len(self.nodes_to_update)+1, numpy.float64)
            counts = numpy.zeros(len(self.nodes_to_update)+1, numpy.int)

            for reaction, distances in self.training_set:
                template = self.reaction_templates[reaction]

                distances_list = [distances[key] for key in distance_keys]
                d = numpy.float64(distances_list[i])
                #dm found by manually summing residuals
                dm = x[-1,i] + sum([x[self.nodes_to_update.index(group),i] for group in template])


                variance = (dm - d)**2

                for group in template:
                    for ancestor in self.group_ancestors[group]:
                        if ancestor not in self.top_nodes:
                            ind = self.nodes_to_update.index(ancestor)
                            variance_sums[ind] += variance
                            counts[ind] += 1
                variance_sums[-1] += variance
                counts[-1] += 1
                
            ci = numpy.zeros(len(counts))

            for j, count in enumerate(counts):
                if count > 2:
                    stdev[j] = numpy.sqrt(variance_sums[j] / (count - 1))
                    ci[j] = scipy.stats.t.ppf(0.975, count - 1) * stdev[j]
                else:
                    stdev[j] = None
                    ci[j] = None

            # Update dictionaries of fitted group values and uncertainties
            for entry in self.all_entries:
                if entry == self.top_nodes[0]:
                    self.groupValues[entry].append(x[-1, i])
                    self.groupUncertainties[entry].append(ci[-1])
                    self.groupCounts[entry].append(counts[-1])
                elif entry.label in [group.label for group in self.nodes_to_update]:
                    index = self.nodes_to_update.index(entry)
                    
                    self.groupValues[entry].append(x[index,i])
                    self.groupUncertainties[entry].append(ci[index])
                    self.groupCounts[entry].append(counts[index])
                else:
                    self.groupValues[entry] = None
                    self.groupUncertainties[entry] = None
                    self.groupCounts[entry] = None
            
            for entry in self.all_entries:
                if self.groupValues[entry] is not None:
                    if not any(numpy.isnan(numpy.array(self.groupUncertainties[entry]))):
                        # should be entry.data.* (e.g. entry.data.uncertainties)
                        uncertainties = numpy.array(self.groupUncertainties[entry])
                        uncertaintyType = '+|-'
                    else:
                        uncertainties = {}
                    # should be entry.*
                    shortDesc = "Fitted to {0} distances.\n".format(self.groupCounts[entry][0])
                    longDesc = "\n".join(self.groupComments[entry])
                    distances_dict = {key:distance for key, distance in zip(distance_keys, self.groupValues[entry])}
                    uncertainties_dict = {key:distance for key, distance in zip(distance_keys, uncertainties)}
                    
                    entry.data = DistanceData(distances=distances_dict, uncertainties=uncertainties_dict)
                    entry.shortDesc = shortDesc
                    entry.longDesc = longDesc
                else:
                    entry.data = DistanceData()
                    entry.longDesc = ''
        logging.info("Finished Updating Entries for {}\n".format(self.family))
        return
        
    def save_database(self, path = None):
        if path is None and self.path is None:
            logging.error("Need path to save output")
        elif path is None:
            path = os.join(self.path, 'TS_groups.py')

        self.database.saveTransitionStateGroups(path)
        logging.info('Saved {} Database to: {}'.format(self.family, path))
        return

In [7]:
families = ['H_Abstraction']#, 'intra_H_migration']
TS_Database_Update(families)
print 'Done'

Done


In [8]:
rmg_database = RMGDatabase()
database_path = os.path.join(os.path.expandvars('$RMGpy'), "..",  'RMG-database', 'input')
rmg_database.load(database_path,
                 kineticsFamilies=['H_Abstraction'],
                 transportLibraries=[],
                 reactionLibraries=[],
                 seedMechanisms=[],
                 thermoLibraries=['primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'CBS_QB3_1dHR' ],
                 solvation=False,
                 )

# TODO: Edit this so it works with multiple databases

ts_database = TransitionStates()
path = os.path.join(os.path.expandvars("$RMGpy"), "..", "AutoTST", "database", "H_Abstraction")
global_context = { '__builtins__': None }
local_context={'DistanceData': DistanceData}
family = rmg_database.kinetics.families["H_Abstraction"]
ts_database.family = family
ts_database.load(path, local_context, global_context)

In [ ]:

ts.groups.loadOldTree("AHHHHHH.py")

In [ ]:
x.database.descendants(x.top_nodes[0])

In [ ]:
count = 0
master_list = []
for value in ts_database.groups.entries.values():
    #print value.data
    
    if not value.data.distances == {}:
        master_list.append(value)
        count += 1
print "Master: {}".format(count)
#master_list

count = 0
second_list = []
for value in x.database.groups.entries.values():
    #print value.data
    
    if not value.data.distances == {}:
        second_list.append(value)
        count += 1
print "Secondary: {}".format(count)
print
#second_list 

for e in master_list:
    if not e in second_list:
        print e
        
for ee in second_list:
    if not ee in master_list:
        print ee

In [ ]:
#ts_database = TransitionStates()
#path = os.path.join(os.path.expandvars("$RMGpy"), "..", "AutoTST", "database", "H_Abstraction")
global_context = { '__builtins__': None }
local_context={'DistanceData': DistanceData}
family = rmg_database.kinetics.families["H_Abstraction"]
ts_database.family = family
t = TSGroups()
t.load("AAAAAAAAAA.py", local_context, global_context)

In [ ]:
x.database.groups.loadOldTree("AHHHHHH.py")
x.save_database("AAAAAAAAAA.py")

In [ ]:
len(x.database.groups.entries)

In [ ]:
count = 0
for value in x.database.groups.entries.values():
    #print value.data
    
    if not (value.data.distances == {}) and not (value.label in master_list):
        print value

In [ ]:
dataPath = os.path.abspath(os.path.join(os.path.dirname(rmgpy.__file__),'..','..','RMG-database', 'input'))
dataPath = os.path.join(dataPath, 'kinetics', 'families', 'H_Abstraction')
#x.save_database(dataPath)
x.save_database("test_please_delete.py")

In [ ]:
for entry, distances in training_set:
    #print entry
    distances = [distances[key] for key in distances]
    
    if distances[0] > distances[1] + distances[2] or distances[0] < 0:
        assert False
        
    if distances[1] > distances[0] + distances[2] or distances[1] < 0:
        assert False
        
    if distances[2] > distances[0] + distances[1] or distances[2] < 0:
        assert False
    

In [ ]:
def get_TS_training_data():
    from autotst.database import DistanceData, TransitionStateDepository, TSGroups, TransitionStates
    rmg_database = RMGDatabase()
    database_path = os.path.join(os.path.expandvars('$RMGpy'), "..",  'RMG-database', 'input')
    rmg_database.load(database_path,
                     kineticsFamilies=['H_Abstraction'],
                     transportLibraries=[],
                     reactionLibraries=[],
                     seedMechanisms=[],
                     thermoLibraries=['primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'CBS_QB3_1dHR' ],
                     solvation=False,
                     )

    ts_database = TransitionStates()
    path = os.path.join(os.path.expandvars("$RMGpy"), "..", "AutoTST", "database", "H_Abstraction")
    global_context = { '__builtins__': None }
    local_context={'DistanceData': DistanceData}
    family = rmg_database.kinetics.families["H_Abstraction"]
    ts_database.family = family
    ts_database.load(path, local_context, global_context)

    # Reaction must be a template reaction... found above

    training_set = [ (entry.item, entry.data.distances) for entry in list(ts_database.depository.entries.itervalues())]
    print "Total Distances Count: {}".format(len(training_set))
    return training_set, ts_database

In [ ]:
def update_indices(database):
    # Updating entry indices based off of tree indices, tree indices are found by descending the tree
    all_entries = []
    top_nodes = database.groups.top
    for top_node in top_nodes:
        descendants = [top_node] + database.groups.descendants(top_node)
        all_entries.extend(descendants)
    #tree_indices = {}
    for tree_index, entry in enumerate(all_entries):
        #tree_indices[entry] = tree_index
        database.groups.entries[entry.label].index = tree_index
        entry.index = tree_index
    print "Tree size: {}".format(len(all_entries))
    #list(ts_database.groups.entries.keys())
    return

# Rewrite not using products, not using top_nodes

In [ ]:
#############################
#All the direct groups we have data for
direct_groups = []
all_reactant_groups = {} #templates organized by reaction

for reaction, distance_data in training_set:
    reactant_groups = []
    #The groups that represent each reactant - also known as the template
    for reactant in reaction.reactants:
        reactant = reactant.molecule[0]
        
        atoms = list(reactant.getLabeledAtoms().itervalues())
        assert atoms is not None
        
        for top_node in top_nodes:
            temp_group = ts_database.groups.descendTree(reactant, atoms, root=top_node)
            if temp_group is not None:
                reactant_group = temp_group
                continue
        
        assert reactant_group is not None
        
        if isinstance(reactant_group, str):
            assert False, "why? {}".format(reactant_group)
            reactant_group = ts_database.groups.entries[reactant_group]
        assert isinstance(reactant_group, Entry)

        reactant_groups.append(reactant_group)        
        direct_groups.append(reactant_group)
        
    all_reactant_groups[reaction] = reactant_groups
    
direct_groups = list(set(direct_groups))
direct_groups.sort(key=lambda x:x.index)

all_ancestors = {} #Key is group, value is its itself and all its ancestors
for direct_group in direct_groups:
    ancestors = [direct_group] + ts_database.groups.ancestors(direct_group)
    for ancestor in ancestors:
        if ancestor in all_ancestors.keys():
            continue
        else:
            all_ancestors[ancestor] = [ancestor] + ts_database.groups.ancestors(ancestor)

nodes_to_update = [group for group in all_ancestors if group not in top_nodes]
nodes_to_update.sort(key=lambda x:x.index)

print 'Nodes to Update: {}'.format(len(nodes_to_update))
print "Reaction Templates: {}".format(len(all_reactant_groups))

In [ ]:
#Test case to see if all_ancestors is working properly
goal = 'C_methane'
entry = None
for item in all_entries:
    #print item.label
    if item.label == goal:
        entry = item
if entry is not None:
    #if isinstance(entry, str):
    #    entry = ts_database.groups.entries[entry]
    print entry
    ancestors = ts_database.groups.ancestors(entry)
print ancestors
print all_ancestors[entry][1:]

In [ ]:
#Stolen from base.py
def getAllCombinations(nodeLists):
    """
    Generate a list of all possible combinations of items in the list of
    lists `nodeLists`. Each combination takes one item from each list
    contained within `nodeLists`. The order of items in the returned lists
    reflects the order of lists in `nodeLists`. For example, if `nodeLists` was
    [[A, B, C], [N], [X, Y]], the returned combinations would be
    [[A, N, X], [A, N, Y], [B, N, X], [B, N, Y], [C, N, X], [C, N, Y]].
    """

    items = [[]]
    for nodeList in nodeLists:
        items = [ item + [node] for node in nodeList for item in items ]

    return items
##############################################
#for entry in ts_database.groups.entries:
#    ts_database.groups.entries[entry].training_data = []


#Attributes of each entry
groupComments = {}; groupCounts = {}; groupUncertainties = {}; groupValues = {}
for entry in all_entries:
    groupComments[entry] = set()
    groupCounts[entry] = []
    groupUncertainties[entry] = []
    groupValues[entry] = []

#distance_keys = sorted(training_set[0][1].keys())
distance_keys = ['d12', 'd13', 'd23']

#all_updated = []
A = []
b = []
for reaction, distance_data in training_set:
    template = all_reactant_groups[reaction]
    distances_list = [distance_data[key] for key in distance_keys]
    #distance_data = numpy.array(distance_list)
    relavent_combinations = []
    for reactant_group in all_reactant_groups[reaction]:
        relavent_combinations.append(all_ancestors[reactant_group])
    assert len(relavent_combinations) == 2 #will throw if reaction does not have 2 reactants
        
    relavent_combinations = getAllCombinations(relavent_combinations)
    #rel_comb is just all combinations of reactant1 and its ancestors with reactant2 and its ancestors
    
    for combination in relavent_combinations:
        Arow = [1 if group in combination else 0 for group in nodes_to_update]
        Arow.append(1) #For use in finding the family component
        #Arow is a binary vector of len(groupList)+1 representing contributing groups to this reaction's distance data
        A.append(Arow)
        b.append(distances_list)
        for group in combination:
            if isinstance(group, str):
                assert False
                group = self.entries[group]

            groupComments[group].add('{0!s}'.format(template))

A = numpy.array(A)
b = numpy.array(b)

#######################################################################
# What I think is going on:
# Groups M and N are associated with a reaction that produces distances, they are differences from the family contribution
# Groups M, N, and the family contribution must add together in some manner to get as close to this distance, as well as in all other reactions of different combinations
# Design matrix of size (All possible group combinations for all possible reactions) by (all groups + 1). Last row is needed for family contribution
    
#distance_keys = sorted(training_set[0][1].distance.keys())
#distance_keys = ['d12', 'd13', 'd23']

x, residuals, rank, s = numpy.linalg.lstsq(A, b)
for i, distance_key in enumerate(distance_keys):
    # Determine error in each group
    variance_sums = numpy.zeros(len(nodes_to_update)+1, numpy.float64)
    stdev = numpy.zeros(len(nodes_to_update)+1, numpy.float64)
    counts = numpy.zeros(len(nodes_to_update)+1, numpy.int)

    #it seems convoluted but it creates a list of variances for all entries in groups entries
    #the last item of the list is the sum of variances for the entire list which represents the entry

    #for index in range(len(trainingSet)):
    #for index, [reaction, distances] in enumerate(trainingSet):
    for reaction, distances in training_set:
        template = all_reactant_groups[reaction]
        
        distances_list = [distance_data[key] for key in distance_keys]
        d = numpy.float64(distances_list[i])
        dm = x[-1,i] + sum([x[nodes_to_update.index(group),i] for group in template]) #if group in nodes_to_update])
        
        #TODO remove:
        for group in template:
            assert group in nodes_to_update
        
        variance = (dm - d)**2
        
        for group in template:
            for ancestor in all_ancestors[group]:
                if ancestor not in top_nodes:
                    ind = nodes_to_update.index(ancestor)
                    variance_sums[ind] += variance
                    counts[ind] += 1
        variance_sums[-1] += variance
        counts[-1] += 1

    import scipy.stats
    ci = numpy.zeros(len(counts))
    #for i in range(len(count)):
    for j, count in enumerate(counts):
        if count > 2:
            stdev[j] = numpy.sqrt(variance_sums[j] / (count - 1))
            ci[j] = scipy.stats.t.ppf(0.975, count - 1) * stdev[j]
            #'probability density function'
        else:
            stdev[j] = None
            ci[j] = None
    
    # Update dictionaries of fitted group values and uncertainties
    for entry in all_entries:
        if entry == top_nodes[0]:
            groupValues[entry].append(x[-1, i])
            groupUncertainties[entry].append(ci[-1])
            groupCounts[entry].append(counts[-1])
        elif entry.label in [group.label for group in nodes_to_update]:
            #index = [group.label for group in groupList].index(entry.label)
            index = nodes_to_update.index(entry)
            groupValues[entry].append(x[index,i])
            groupUncertainties[entry].append(ci[index])
            groupCounts[entry].append(counts[index])
        else:
            groupValues[entry] = None
            groupUncertainties[entry] = None
            groupCounts[entry] = None

In [ ]:
for entry in all_entries:
    print entry.label
    print groupValues[entry]
    print groupUncertainties[entry]
    print len(groupComments[entry])
    break
print
for entry in all_entries:
    if entry.index == 5:
        print entry.label
        print groupValues[entry]
        print groupUncertainties[entry]
        print len(groupComments[entry])
        break

"""neg = []
all_neg = []
impossible = []
for entry in groupValues:
    distances = groupValues[entry]
    uncertainties = groupUncertainties[entry]
    if distances is not None:
        has_neg = False
        has_all_neg = False
        #Impossible ones have negative distances and uncertainties too small to create a range with positive values
        is_impossible = False
        
        if distances[0] < 0 or distances[1] < 0 or distances[2] < 0:
            has_neg = True
        
        if distances[0] < 0 and distances[1] < 0 and distances[2] < 0:
            has_all_neg = True
        
        if distances[0] < 0 and distances[0] + uncertainties[0] < 0:
            is_impossible = True
        
        if distances[1] < 0 and distances[1] + uncertainties[1] < 0:
            is_impossible = True
        
        if distances[2] < 0 and distances[2] + uncertainties[2] < 0:
            is_impossible = True
        
        if has_neg:
            neg.append(entry)
        if has_all_neg:
            all_neg.append(entry)
        if is_impossible:
            impossible.append(entry)
print
print "Atleast one negative: {}".format(len(neg))
print "All negative: {}".format(len(all_neg))
print "Negative and too small uncertainty: {}".format(len(impossible))"""

print
print nodes_to_update[13]
print counts[13]
print ci[13]

In [ ]:
count = 0
for entry in groupUncertainties:
    if groupUncertainties[entry] is not None:
        count += 1
        print "{}   \t:\t{}".format(entry, groupUncertainties[entry])
print
print count

In [ ]:
atomList = []
for top_node in top_nodes:
    group = top_node.item
    print group
    if isinstance(top_node.item, LogicNode):
        group = top_node.item.getPossibleStructures(ts_database.groups.entries)[0]
    top_node_atoms = group.getLabeledAtoms()
    atomList.extend([top_node_atoms])

atomList = atomList[1]
atomList

In [ ]:
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################

# Using top_nodes, no products

In [ ]:
#############################
#All the direct groups we have data for
direct_groups = []
all_reactant_groups = {} #templates organized by reaction

for reaction, distance_data in training_set:
    reactant_groups = []
    #The groups that represent each reactant - also known as the template
    for reactant in reaction.reactants:
        reactant = reactant.molecule[0]
        
        atoms = list(reactant.getLabeledAtoms().itervalues())
        assert atoms is not None
        
        for top_node in top_nodes:
            temp_group = ts_database.groups.descendTree(reactant, atoms, root=top_node)
            if temp_group is not None:
                reactant_group = temp_group
                continue
        
        assert reactant_group is not None
        
        if isinstance(reactant_group, str):
            assert False, "why? {}".format(reactant_group)
            reactant_group = ts_database.groups.entries[reactant_group]
        assert isinstance(reactant_group, Entry)

        reactant_groups.append(reactant_group)        
        direct_groups.append(reactant_group)
        
    all_reactant_groups[reaction] = reactant_groups
    
direct_groups = list(set(direct_groups))
direct_groups.sort(key=lambda x:x.index)

all_ancestors = {} #Key is group, value is its itself and all its ancestors
for direct_group in direct_groups:
    ancestors = [direct_group] + ts_database.groups.ancestors(direct_group)
    for ancestor in ancestors:
        if ancestor in all_ancestors.keys():
            continue
        else:
            all_ancestors[ancestor] = [ancestor] + ts_database.groups.ancestors(ancestor)

nodes_to_update = [group for group in all_ancestors]
nodes_to_update.sort(key=lambda x:x.index)

print 'Nodes to Update: {}'.format(len(nodes_to_update))
print "All Reactant Groups: {}".format(len(all_reactant_groups))

In [ ]:
##############################################
#for entry in ts_database.groups.entries:
#    ts_database.groups.entries[entry].training_data = []

#Attributes of each entry
groupComments = {}; groupCounts = {}; groupUncertainties = {}; groupValues = {}
for entry in all_entries:
    groupComments[entry] = set()
    groupCounts[entry] = []
    groupUncertainties[entry] = []
    groupValues[entry] = []

#distance_keys = sorted(training_set[0][1].keys())
distance_keys = ['d12', 'd13', 'd23']

#all_updated = []
A = []
B = []
for reaction, distance_data in training_set:
    template = all_reactant_groups[reaction]
    distances_list = [distance_data[key] for key in distance_keys]
    #distance_data = numpy.array(distance_list)
    relavent_combinations = []
    for reactant_group in all_reactant_groups[reaction]:
        relavent_combinations.append(all_ancestors[reactant_group])
    assert len(relavent_combinations) == 2 #will throw if reaction does not have 2 reactants
        
    relavent_combinations = getAllCombinations(relavent_combinations)
    #rel_comb is just all combinations of reactant1 and its ancestors with reactant2 and its ancestors
    
    for combination in relavent_combinations:
        Arow = [1 if group in combination else 0 for group in nodes_to_update]
        Arow.append(1)
        #Arow is a binary vector of len(groupList)+1 representing contributing groups to this reaction's distance data
        A.append(Arow)
        B.append(distances_list)
        for group in combination:
            if isinstance(group, str):
                assert False
                group = self.entries[group]

            groupComments[group].add('{0!s}'.format(template))
print "Length of A: {}".format(len(A))
print

A = numpy.array(A)
B = numpy.array(B)

#######################################################################
# What I think is going on:
# Groups M and N are associated with a reaction that produces distances
# Groups M and N must add together in some manner to get as close to this distance as well as in all other reactions of different combinations
# Design matrix of size (All possible group combinations for all possible reactions) by (all groups + 1). Last row is needed for intercept
    
#distance_keys = sorted(training_set[0][1].distance.keys())
#distance_keys = ['d12', 'd13', 'd23']

x, residuals, rank, s = numpy.linalg.lstsq(A, B)
for i, distance_key in enumerate(distance_keys):
    # Determine error in each group
    variance_sums = numpy.zeros(len(nodes_to_update)+1, numpy.float64)
    stdev = numpy.zeros(len(nodes_to_update)+1, numpy.float64)
    counts = numpy.zeros(len(nodes_to_update)+1, numpy.int)

    #it seems convoluted but it creates a list of variances for all entries in groups entries
    #the last item of the list is the sum of variances for the entire list which represents the entry

    #for index in range(len(trainingSet)):
    #for index, [reaction, distances] in enumerate(trainingSet):
    for reaction, distances in training_set:
        template = all_reactant_groups[reaction]
        
        distances_list = [distance_data[key] for key in distance_keys]
        d = numpy.float64(distances_list[i])
        dm = x[-1,i] + sum([x[nodes_to_update.index(group),i] for group in template if group in nodes_to_update])
        
        variance = (dm - d)**2
        
        for group in template:
            for ancestor in all_ancestors[group]:
                #if ancestor not in top_nodes:
                ind = nodes_to_update.index(ancestor)
                variance_sums[ind] += variance
                counts[ind] += 1
        variance_sums[-1] += variance
        counts[-1] += 1

    #import scipy.stats
    ci = numpy.zeros(len(counts))
    #for i in range(len(count)):
    for j, count in enumerate(counts):
        if count > 1:
            stdev[j] = numpy.sqrt(variance_sums[j] / (count - 1))
            ci[j] = scipy.stats.t.ppf(0.975, count - 1) * stdev[j]
            #'probability density function'
        else:
            stdev[j] = None
            ci[j] = None
    
    # Update dictionaries of fitted group values and uncertainties
    for entry in all_entries:
        """if entry == top_nodes[0]:
            groupValues[entry].append(x[-1, i])
            groupUncertainties[entry].append(ci[-1])
            groupCounts[entry].append(counts[-1])"""
        if entry.label in [group.label for group in nodes_to_update]:
            #index = [group.label for group in groupList].index(entry.label)
            index = nodes_to_update.index(entry)
            groupValues[entry].append(x[index,i])
            groupUncertainties[entry].append(ci[index])
            groupCounts[entry].append(counts[index])
        else:
            groupValues[entry] = None
            groupUncertainties[entry] = None
            groupCounts[entry] = None

for entry in all_entries:
    print entry.label
    print groupValues[entry]
    print groupUncertainties[entry]
    print len(groupComments[entry])
    break
print
for entry in all_entries:
    if entry.index == 5:
        print entry.label
        print groupValues[entry]
        print groupUncertainties[entry]
        print len(groupComments[entry])
        break
        
has_neg_distances = [entry for entry in all_entries if value < 0 for value in groupValues[entry]]
print len(has_neg_distances)

# Experimental

In [ ]:
#############################
#All the direct groups we have data for
direct_groups = []
all_reactant_groups = {} #templates organized by reaction

for reaction, distance_data in training_set:
    reactant_groups = []
    #The groups that represent each reactant - also known as the template
    for reactant in reaction.reactants:
        reactant = reactant.molecule[0]
        
        atoms = list(reactant.getLabeledAtoms().itervalues())
        assert atoms is not None
        
        for top_node in top_nodes:
            temp_group = ts_database.groups.descendTree(reactant, atoms, root=top_node)
            if temp_group is not None:
                reactant_group = temp_group
                continue
        
        assert reactant_group is not None
        
        if isinstance(reactant_group, str):
            assert False, "why? {}".format(reactant_group)
            reactant_group = ts_database.groups.entries[reactant_group]
        assert isinstance(reactant_group, Entry)

        reactant_groups.append(reactant_group)        
        direct_groups.append(reactant_group)
        
    all_reactant_groups[reaction] = reactant_groups
    
direct_groups = list(set(direct_groups))
direct_groups.sort(key=lambda x:x.index)

all_ancestors = {} #Key is group, value is its itself and all its ancestors
for direct_group in direct_groups:
    ancestors = [direct_group] + ts_database.groups.ancestors(direct_group)[:-1]
    for ancestor in ancestors:
        if ancestor in all_ancestors.keys():
            continue
        else:
            all_ancestors[ancestor] = [ancestor] + ts_database.groups.ancestors(ancestor)[:-1]

nodes_to_update = [group for group in all_ancestors if group not in top_nodes]
nodes_to_update.sort(key=lambda x:x.index)

print 'Nodes to Update: {}'.format(len(nodes_to_update))
print "All Reactant Groups: {}".format(len(all_reactant_groups))

In [ ]:
##############################################
#for entry in ts_database.groups.entries:
#    ts_database.groups.entries[entry].training_data = []

#Attributes of each entry
groupComments = {}; groupCounts = {}; groupUncertainties = {}; groupValues = {}
for entry in all_entries:
    groupComments[entry] = set()
    groupCounts[entry] = []
    groupUncertainties[entry] = []
    groupValues[entry] = []

#distance_keys = sorted(training_set[0][1].keys())
distance_keys = ['d12', 'd13', 'd23']

#all_updated = []
A = []
B = []
for reaction, distance_data in training_set:
    template = all_reactant_groups[reaction]
    distances_list = [distance_data[key] for key in distance_keys]
    #distance_data = numpy.array(distance_list)
    relavent_combinations = []
    for reactant_group in all_reactant_groups[reaction]:
        relavent_combinations.append(all_ancestors[reactant_group])
    assert len(relavent_combinations) == 2 #will throw if reaction does not have 2 reactants
        
    relavent_combinations = getAllCombinations(relavent_combinations)
    #rel_comb is just all combinations of reactant1 and its ancestors with reactant2 and its ancestors
    
    for combination in relavent_combinations:
        Arow = [1 if group in combination else 0 for group in nodes_to_update]
        #Arow.append(1)
        #Arow is a binary vector of len(groupList)+1 representing contributing groups to this reaction's distance data
        A.append(Arow)
        B.append(distances_list)
        for group in combination:
            if isinstance(group, str):
                assert False
                group = self.entries[group]

            groupComments[group].add('{0!s}'.format(template))

print "Length of A: {}".format(len(A))
print
            
A = numpy.array(A)
B = numpy.array(B)

#######################################################################
# What I think is going on:
# Groups M and N are associated with a reaction that produces distances
# Groups M and N must add together in some manner to get as close to this distance as well as in all other reactions of different combinations
# Design matrix of size (All possible group combinations for all possible reactions) by (all groups + 1). Last row is needed for intercept
    
#distance_keys = sorted(training_set[0][1].distance.keys())
#distance_keys = ['d12', 'd13', 'd23']

x, residuals, rank, s = numpy.linalg.lstsq(A, B)
#x, residual = nnls(A, B)
for i, distance_key in enumerate(distance_keys):
    # Determine error in each group
    variance_sums = numpy.zeros(len(nodes_to_update)+1, numpy.float64)
    stdev = numpy.zeros(len(nodes_to_update)+1, numpy.float64)
    counts = numpy.zeros(len(nodes_to_update)+1, numpy.int)

    #it seems convoluted but it creates a list of variances for all entries in groups entries
    #the last item of the list is the sum of variances for the entire list which represents the entry

    #for index in range(len(trainingSet)):
    #for index, [reaction, distances] in enumerate(trainingSet):
    for reaction, distances in training_set:
        template = all_reactant_groups[reaction]
        
        distances_list = [distance_data[key] for key in distance_keys]
        d = numpy.float64(distances_list[i])
        dm = x[-1,i] + sum([x[nodes_to_update.index(group),i] for group in template if group in nodes_to_update])
        
        variance = (dm - d)**2
        
        for group in template:
            for ancestor in all_ancestors[group]:
                if ancestor not in top_nodes:
                    ind = nodes_to_update.index(ancestor)
                    variance_sums[ind] += variance
                    counts[ind] += 1
        variance_sums[-1] += variance
        counts[-1] += 1

    #import scipy.stats
    ci = numpy.zeros(len(counts))
    #for i in range(len(count)):
    for j, count in enumerate(counts):
        if count > 1:
            stdev[j] = numpy.sqrt(variance_sums[j] / (count - 1))
            ci[j] = scipy.stats.t.ppf(0.975, count - 1) * stdev[j]
            #'probability density function'
        else:
            stdev[j] = None
            ci[j] = None
    
    # Update dictionaries of fitted group values and uncertainties
    for entry in all_entries:
        if entry == top_nodes[0]:
            groupValues[entry].append(x[-1, i])
            groupUncertainties[entry].append(ci[-1])
            groupCounts[entry].append(counts[-1])
        elif entry.label in [group.label for group in nodes_to_update]:
            #index = [group.label for group in groupList].index(entry.label)
            index = nodes_to_update.index(entry)
            groupValues[entry].append(x[index,i])
            groupUncertainties[entry].append(ci[index])
            groupCounts[entry].append(counts[index])
        else:
            groupValues[entry] = None
            groupUncertainties[entry] = None
            groupCounts[entry] = None

In [ ]:
for entry in all_entries:
    print entry.label
    print groupValues[entry]
    print groupUncertainties[entry]
    print len(groupComments[entry])
    break
print
for entry in all_entries:
    if entry.index == 5:
        print entry.label
        print groupValues[entry]
        print groupUncertainties[entry]
        print len(groupComments[entry])
        break

neg = []
all_neg = []
impossible = []
for entry in groupValues:
    distances = groupValues[entry]
    uncertainties = groupUncertainties[entry]
    if distances is not None:
        has_neg = False
        has_all_neg = False
        #Impossible ones have negative distances and uncertainties too small to create a range with positive values
        is_impossible = False
        
        if distances[0] < 0 or distances[1] < 0 or distances[2] < 0:
            has_neg = True
        
        if distances[0] < 0 and distances[1] < 0 and distances[2] < 0:
            has_all_neg = True
        
        if distances[0] < 0 and distances[0] + uncertainties[0] < 0:
            is_impossible = True
        
        if distances[1] < 0 and distances[1] + uncertainties[1] < 0:
            is_impossible = True
        
        if distances[2] < 0 and distances[2] + uncertainties[2] < 0:
            is_impossible = True
        
        if has_neg:
            neg.append(entry)
        if has_all_neg:
            all_neg.append(entry)
        if is_impossible:
            impossible.append(entry)
print
print "Atleast one negative: {}".format(len(neg))
print "All negative: {}".format(len(all_neg))
print "Negative and too small unvertainty: {}".format(len(impossible))

In [ ]:
A = []
for i in range(5):
    A_row = []
    for j in range(5):
        y = 0
        if i == j:
            y = 2
        A_row.append(y)
    A.append(A_row)
    
A = numpy.array(A)
B = range(5,10)
x, residuals = nnls(A,B)
x

In [ ]:
A = 2*numpy.random.rand(10,3)+1
A

In [ ]:
A[:,0]

In [ ]:
B = numpy.array
B = numpy.zeros((10,3))
B[:,0] = (A[:,1]**2+A[:,2]**2-A[:,0]**2)/(2*A[:,1])
B[:,2] = (A[:,0]**2+A[:,1]**2-A[:,2]**2)/(2*A[:,1])
B[:,1] = numpy.sqrt(A[:,2]**2 - B[:,0]**2)
#B = 2*B
B